In [131]:
%load_ext autoreload
%autoreload 2
import recordlinkage as rl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [169]:
from amd_bkg_matchids import *

# 1. fetch all unmatched Amadeus hotels
print("Loading Amadeus hotels")
amdh = load_amadeus()

# 2. fetch all unmatched Booking.com hotels
print("Loading Booking hotels")
bkgh = load_booking()

Loading Amadeus hotels
Loading Booking hotels


In [182]:
# 3. load existing matches
matches = load_matches()

In [183]:
from copy import deepcopy
matches_ = deepcopy(matches)

In [171]:
amdh_ = amdh.copy()

In [172]:
# 5. Build geo index
print("Building Geo Index")
geo_index = GeoGridIndex()
for i, hb in bkgh.iterrows():
    geo_index.add_point(GeoPoint(hb['lat'], hb['lng'], ref=hb))

Building Geo Index


In [ ]:
amdh = amdh_.sample(10)
matches

In [ ]:
amdh = amdh_.sample(10)

In [187]:
matches = {a: b for (a,b) in matches_.items() if a in amdh.amd_id.values}

In [188]:
len(matches)

3

In [178]:
cand_data = generate_comparison_vectors(amdh, geo_index, radius=3, nsim_threshold=0.7)

In [179]:
len(cand_data)

94

In [185]:
match_pairs = pd.MultiIndex.from_tuples(matches.items(), names=['amd_id', 'bkg_id'])

In [186]:
len(match_pairs)

3

In [165]:
match_pairs = generate_match_pairs()

In [88]:
cand_pairs = pd.MultiIndex.from_tuples(cand_pair_ids, names=['amd_id', 'bkg_id'])

In [89]:
len(cand_pairs)

81732

In [90]:
cand_data = pd.DataFrame(features, index=cand_pairs, columns=['nsim1','nsim2','dist'])

In [134]:
cand_data = pd.read_csv('/mnt/xvdb/projects/idmatching/cand_data.csv', index_col=[0,1])

In [144]:
len(cand_data)

4079973

In [164]:
cand_data = cand_data.sample(5000)

In [166]:
match_pairs = cand_data.index & match_pairs

In [167]:
len(cand_data)

5000

In [168]:
len(match_pairs)

3

In [160]:
from recordlinkage.datasets import load_krebsregister
krebs_data, krebs_match = load_krebsregister()

In [161]:
golden_pairs = krebs_data[0:5000]
golden_matches_index = golden_pairs.index & krebs_match

In [163]:
golden_matches_index

MultiIndex(levels=[[5, 67, 73, 113, 171, 186, 189, 237, 276, 279, 405, 424, 432, 469, 501, 520, 597, 602, 627, 712, 757, 801, 814, 840, 855, 906, 984, 997, 1006, 1056, 1091, 1092, 1223, 1285, 1373, 1396, 1407, 1419, 1443, 1465, 1596, 1763, 1806, 1814, 1821, 1857, 1897, 1910, 1933, 1955, 2060, 2086, 2157, 2188, 2204, 2300, 2375, 2401, 2422, 2429, 2440, 2481, 2491, 2518, 2628, 2638, 2644, 2646, 2670, 2697, 2699, 2700, 2702, 2726, 2760, 2833, 2836, 2855, 2918, 2935, 2942, 3081, 3130, 3172, 3191, 3193, 3197, 3213, 3226, 3238, 3298, 3318, 3327, 3357, 3364, 3440, 3585, 3621, 3649, 3710, 3751, 3791, 3794, 3995, 4045, 4175, 4218, 4238, 4247, 4254, 4308, 4346, 4375, 4395, 4402, 4407, 4412, 4483, 4486, 4520, 4526, 4602, 4608, 4674, 4688, 4712, 4767, 4833, 4843, 4914, 4949, 5098, 5172, 5221, 5232, 5237, 5244, 5329, 5363, 5387, 5449, 5510, 5536, 5541, 5560, 5571, 5586, 5591, 5599, 5607, 5613, 5689, 5736, 5771, 5795, 5801, 5821, 5938, 5947, 5963, 6075, 6084, 6355, 6389, 6472, 6484, 6502, 6542, 6583

# K-means

In [136]:
kmeans = rl.KMeansClassifier()
result_kmeans = kmeans.learn(cand_data)

In [137]:
# The predicted number of matches
len(result_kmeans)

1799710

In [138]:
cm_kmeans = rl.confusion_matrix(match_pairs, result_kmeans, len(cand_data))
fscore_kmeans = rl.fscore(cm_kmeans)

fscore_kmeans

0.001795752448284919

# Logistic Regression

In [189]:
logreg = rl.LogisticRegressionClassifier()
logreg.learn(cand_data, match_pairs)

# Predict the match status for all record pairs
result_logreg = logreg.predict(cand_data)

len(result_logreg)

0

In [190]:
cand_data.head()

nsim1     nsim2      dist
amd_id    bkg_id                               
700113830 1614468  0.160000  0.705882  1.249443
          1614428  0.153846  0.705882  1.249555
          1614446  0.240000  0.705882  1.249318
          1614433  0.153846  0.705882  1.249243
          1614448  0.240000  0.705882  1.249331

In [140]:
conf_logreg = rl.confusion_matrix(match_pairs, result_logreg, len(cand_data))
conf_logreg

array([[      0,   92533],
       [      0, 3987440]])

In [141]:
rl.fscore(conf_logreg)

/home/ubuntu/projects/livingrooms/venv/lib/python3.5/site-packages/recordlinkage/measures.py:149: RuntimeWarning: invalid value encountered in long_scalars
  / (confusion_matrix[0, 0] + confusion_matrix[1, 0])


nan

# Support Vector Machines

In [153]:
# Train the classifier
svm = rl.SVMClassifier()
svm.learn(cand_data, match_pairs)

# Predict the match status for all record pairs
result_svm = svm.predict(cand_data)

len(result_svm)

0

In [143]:
conf_svm = rl.confusion_matrix(match_pairs, result_svm, len(cand_data))
conf_svm

array([[      0,   92533],
       [      0, 3987440]])

In [100]:
rl.fscore(conf_svm)

0.8167524697370251

# Expectation/Conditional Maximization Algorithm

In [102]:
cand_data_ecm = cand_data.copy()

In [104]:
cand_data_ecm.nsim1 = (cand_data.nsim1 * 100).astype(int)

In [106]:
cand_data_ecm.nsim2 = (cand_data.nsim2 * 100).astype(int)

In [110]:
cand_data_ecm.dist = (cand_data.dist * 100).astype(int)

In [117]:
cand_data_ecm.fillna(0, inplace=True)

In [ ]:
ecm = rl.ECMClassifier()
result_ecm = ecm.learn(cand_data_ecm)

len(result_ecm)

In [ ]:
conf_ecm = rl.confusion_matrix(match_pairs, result_ecm, len(cand_data))
conf_ecm

In [ ]:
rl.fscore(conf_ecm)